In [7]:
import os
import sys
import pandas as pd
import numpy as np
import requests
import re
import io
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Okt 
import time
import json
import csv

In [8]:
browser = Chrome()
browser.maximize_window()
base_url = 'https://pann.nate.com'
browser.get(base_url)

In [9]:
#톡커들의 선택 명예의 전당 - 일상톡 랭킹 - 실시간 (당일 실시간 베스트 게시물)
browser.find_elements_by_xpath('//*[@id="container"]/div[5]/div[2]/div[2]/div[1]/a')[0].click()
browser.find_elements_by_xpath('//*[@id="tselect"]/a')[0].click()
browser.find_elements_by_xpath('//*[@id="container"]/div[3]/div[1]/div[2]/div[2]/ul/li[2]/a')[0].click()

In [4]:
res = requests.get(browser.current_url)
res.raise_for_status()
res.encoding = None
html = res.text

soup = BeautifulSoup(html,'html.parser')

titles = []
links = []

first_list = soup.find('div', {'class': 'cntList'}).findAll('li')

for li in first_list: 
    f_link = li.findAll('a')
    for a in f_link:
        real_link = 'https://pann.nate.com' + a.get('href')  
    links.append(real_link)
    
for li in first_list:
    f_title = li.findAll('dl')
    for dl in f_title:
        t = dl.find('a')
        real_title = t.get('title')
    titles.append(real_title)
    
browser.find_elements_by_xpath('//*[@id="last"]')[0].click()

res = requests.get(browser.current_url)
res.raise_for_status()
res.encoding = None
html = res.text

soup = BeautifulSoup(html,'html.parser')

second_list = soup.find('div', {'class': 'cntList'}).findAll('li')

for li in second_list: 
    s_link = li.findAll('a')
    for a in s_link:
        real_link = 'https://pann.nate.com' + a.get('href')  
    links.append(real_link)

for li in second_list:
    s_title = li.findAll('dl')
    for dl in s_title:
        t = dl.find('a')
        real_title = t.get('title')
    titles.append(real_title)

print({'제목': titles, 'url': links}, sep='\n')

{'제목': ['다른 여자랑 계속 출퇴근 같이하겠다는 남편', "'영남대 강간을 덮으려 한다' 센터입장과 사실확인", '눈수술로 사망사고낸 가해자 여자였으면', '한국이 별일 다 터져도 안망하는이유', '이렇게 당당한 태도. 내일 고소합니다.', '한강 의대생 동기 폭탄 발언 “니가 죽인거야”', '보태준거 없이 바라는 시댁 어떻게 방어하죠', '아이를 과하게 띄워 주며 애지중지 하던 부모', '술만마시면 우리집에오는시어머니', '신당동에서 구찌 신발 맡겼다가 훼손이 되었습니다. 2', '근데 솔직히 트와이스가 대단하긴함', '꼭 형님하고 비슷하게 해가야하나요?', '오늘킹덤', '교환 학생으로 한국에서 강의 듣던 일본녀.', '[만화] 우리나라 현실', '페미니스트=정신병 확실한 이유', '남편 전여친의 모친상 가라고 해야할까요?', '강아지가 서운함을 느끼는 순간..', '마른 여자의 식단', '전남친 결혼한다고 카톡왔어', '일반인과 우울증 환자의 차이', '진짜 한강의대생사건 개에에에소름부분', '(방탈죄송)(긴글 주의)신혼인데 저희 부모님때문에 답...', '치킨무에 대한 일본반응', '한강 의대생 살인사건인거 확실해요??', '일전에 시어머니가 백명이라고 고민올렸던 사람입니다...', '자기 애견추가비용 안내는 친구 대처방법', '놀이터 미끄럼틀에서 어떤아이가 유리병 덩져서 파편...', '김밥 먹고 임플란트 하게 됐어요', '다음 생에는 다이아수저집 개로 태어나', '안친한 남사친이 밥먹자 하면', '그립긴한데', '스타1 프로게이머였던 이영호가 공익으로 간 이유', '외도 의심해 아내 sns 열어본 남편...', '(추가)결혼하자는게 문제인가요?', '요즘 사주는 요즘 기준으로 봐야 한다', '어리고 예쁜 아내와 살고있는걱  후기', '한강 의대생 그냥 친구가 범인이길 바라는거 같음', '백지에 함부로 싸인하면 안되는 이유', '민트초코 발명', '사회생활 잘하고싶은데.... 이리저리 휘둘려서 너무 ...', '다시 만나면 진짜 잘할자신있는데..

In [6]:
txt = []
    
for i in links:
    try: 
        #본문
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None            
        html2 = res.text
        
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "viewarea"})            
        parags = contentArea.findAll("div", {"id" : "contentArea"})

        content = ""

        for parag in parags:
            content += parag.text.replace('&nbsp;| |\t|\r|\n|', '')
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp; | &nbsp;| \n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', content)     
        content = re.sub('<[^>]+>','',content)

        txt.append(content)
             
    except HTTPError as e:
        txt.append('')
    except URLError as e:
        txt.append('')
    except AttributeError as e:
        txt.append('')
                      

okt=Okt()
title_morphs = []
txt_morphs = []
    
for i in titles:
    title_morphs.append(okt.morphs(i))
    
for i in txt:
    txt_morphs.append(okt.morphs(i))
             
    
nate_dict = {
    '제목' : titles,
    '본문' : txt
}


df = pd.DataFrame(nate_dict) 
df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")
    
morphs_dict = {
    '제목 형태소' : title_morphs,
    '본문 형태소' : txt_morphs
}

df2 = pd.DataFrame(morphs_dict)
df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")
    
#browser.close()